# Train an IntegratedML model on Readmission Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [3]:
!conda activate
!pip3 install jupyterthemes

/bin/bash: /opt/miniconda3/bin/conda: Permission denied
Could not fetch URL https://pypi.org/simple/jupyterthemes/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/jupyterthemes/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping
ERROR: Could not find a version that satisfies the requirement jupyterthemes (from versions: none)
ERROR: No matching distribution found for jupyterthemes


In [2]:
!jt -r

Reset css and font defaults in:
/root/.jupyter/custom &
/root/.local/share/jupyter/nbextensions


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

!ls /tf/intersystems-jdbc-3.1.0.jar
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

In [4]:
!pip install JayDeBeApi

In [5]:
!pip list JPype1

Package                Version
---------------------- -----------
absl-py                0.9.0
asn1crypto             0.24.0
astunparse             1.6.3
attrs                  19.3.0
backcall               0.1.0
bleach                 3.1.5
cachetools             4.1.0
certifi                2020.4.5.1
chardet                3.0.4
cryptography           2.1.4
cycler                 0.10.0
decorator              4.4.2
defusedxml             0.6.0
entrypoints            0.3
gast                   0.3.3
google-auth            1.14.2
google-auth-oauthlib   0.4.1
google-pasta           0.2.0
grpcio                 1.28.1
h5py                   2.10.0
idna                   2.6
importlib-metadata     1.6.0
ipykernel              5.1.1
ipython                7.14.0
ipython-genutils       0.2.0
ipywidgets             7.5.1
JayDeBeApi             1.2.3
jedi                   0.17.0
Jinja2                 2.11.2
joblib                 0.15.1
JPype1                 0.7.5
jsonschema             3

### 1. Set environment variables, if necessary

In [ ]:
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

### 2. Get jdbc connection and cursor

In [6]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:1972/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [7]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

### 3. specify the source data table

In [9]:
dataTable = 'SQLUser.Readmission'

### 4. Execute a query and display results in Pandas DataFrame

In [10]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

,ID,AdmitReason,Com_ANY_MALIGNANCY_INCLUDING_LYMPHOMA_AND_LEUKEMIA_EXCEPT_MALIGNANT_NEOPLASM_OF_SKIN,Com_CEREBROVASCULAR_DISEASE,Com_CHRONIC_PULMONARY_DISEASE,Com_DEMENTIA,Com_DIABETES_WITHOUT_CHRONIC_COMPLICATION,Com_HEMIPLEGIA_OR_PARAPLEGIA,Com_MILD_LIVER_DISEASE,Com_MYOCARDIAL_INFARCTION,...,MxNumAdmitsSixMonth,MxNumAdmitsThreeMonth,MxNumAdmitsTwelveMonth,MxNumDiagOneMonth,MxNumDiagTwelveMonth,MxSmoker,MxStartDateDayOfMonth,MxStartDateDayOfWeek,MxStartDateMonth,MxWillReAdmit
0,1,SNOMED Code,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,10,7,7,0
1,2,SNOMED Code,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,10,2,12,0
2,3,SNOMED Code,0,0,0,0,0,0,0,0,...,0,0,0,1,2,0,1,3,10,0
3,4,SNOMED Code,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,31,5,8,1
4,5,SNOMED Code,0,0,0,0,0,0,0,0,...,1,1,1,1,2,0,4,2,9,0
5,6,Encounter Inpatient,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,12,5,8,0
6,7,SNOMED Code,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,6,8,1
7,8,SNOMED Code,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,12,6,8,0
8,9,Encounter Inpatient,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,11,3,3,0
9,10,Encounter Inpatient,0,0,0,0,0,0,0,0,...,0,0,1,1,2,0,15,2,9,0


In [11]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

,Aggregate_1
0,14047


### 5. Make some views to split training and testing datasets

In [ ]:
df1 = pd.read_sql("SELECT COUNT(*) FROM %s WHERE  ID>=11000" % dataTable, conn)
display(df1)

In [12]:
# Training set view
curs.execute("CREATE VIEW SQLUser.ReadmissionTraining AS SELECT * FROM SQLUser.Readmission WHERE ID<11000")
# Prediction set
curs.execute("CREATE VIEW SQLUser.ReadmissionPredict AS SELECT * FROM SQLUser.Readmission WHERE ID>=11000")

DatabaseError: java.sql.SQLException: [SQLCODE: <-201>:<Table or view name not unique>]
[Location: <ServerLoop>]

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [14]:
curs.execute("DROP MODEL ReadmissionModel")
curs.execute("CREATE MODEL ReadmissionModel PREDICTING (MxWillReAdmit) FROM ReadmissionTraining")

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [15]:
curs.execute("TRAIN MODEL ReadmissionModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [16]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,ReadmissionModel,ReadmissionModel2,AutoML,2020-07-24 14:51:55.344000,classification,"ModelType:Random Forest, Package:sklearn, Prob..."


### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [17]:
df4 = pd.read_sql("SELECT PREDICT(ReadmissionModel) AS PredictedReadmit, \
                  MxWillReAdmit AS ActualReadmit FROM ReadmissionPredict", conn)
display(df4)

,PredictedReadmit,ActualReadmit
0,1,1
1,1,1
2,0,0
3,1,0
4,1,0
...,...,...
3043,0,0
3044,0,0
3045,0,0
3046,0,0


### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [18]:
curs.execute("VALIDATE MODEL ReadmissionModel FROM SQLUser.ReadmissionPredict")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
ReadmissionModel21,0.82,0.59,0.63,0.55
